# LIME

LIME is 'locally interpetable model-agnostic explanations'.

The paper is at http://arxiv.org/pdf/1602.04938v1.pdf , by Ribeiro, Singh, and Guestrin.  Ribeiro has a blog post about it at https://homes.cs.washington.edu/~marcotcr/blog/lime/ . There is code provided by Ribeiro at https://github.com/marcotcr/lime

There is thus already ample documentation and code about LIME, and this repo is for self-study purposes primarily, and likely wont introduce anything much new to the world, for now :-)

## What LIME does

LIME does the following:
- creates interpretable features, which for sparse nlp models means, they draw the first few features from a LARS path, and use those.  self-study notebook for LARS: https://github.com/hughperkins/selfstudy-LARS/blob/master/test_lars.ipynb
- samples from interpretable feature space, near an example we wish to explain
- uses local gradients, from near the target example, to explain which interpretable features most affect decisions around that example

## LIME Experiments

### Train and test distributions differ

- train on news20, for atheist vs christianity
- test against new [religion](https://github.com/marcotcr/lime-experiments/blob/master/religion_dataset.tar.gz) dataset, created from websites from from [DMOZ](https://github.com/marcotcr/lime-experiments/blob/master/religion_dataset.tar.gz) directory
  - these data points have similar classes to the news20 training sets, ie atheism vs christianity.  However, the features are fairly different, and eg learning the names of prolific atheist posters in news20 wont generalize to the DMOZ websites.
- the idea is to examine to what extent the LIME explanations (or any other explanations for that matter) can facilitate rmeoving 'junk' features, after/during training, and thus improving the score on the DMOZ-derived dataset